Import libraries:

In [4]:
from __future__ import unicode_literals
from nltk.tokenize import word_tokenize
import wikipediaapi
import string

import pandas as pd
import fuzzy_pandas as fpd
from collections import Counter
from webdriver_manager.chrome import ChromeDriverManager

from difflib import SequenceMatcher
    
import os
from tqdm import tqdm
from docx import Document
from docx.shared import Cm
from docx.shared import Pt
import concurrent.futures
from flask import Flask, render_template, request
from docx.enum.dml import MSO_THEME_COLOR_INDEX
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt
import docx
import time

from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER
from reportlab.platypus import Table, TableStyle
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.platypus import Image
from reportlab import platypus

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

import numpy as np
from tika import parser
from gensim.summarization.summarizer import summarize

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from requests import get
import random
import html
import re

import nltk
import collections
nltk.download('punkt')
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4.element import Comment

import networkx as nx
from nltk.tokenize import sent_tokenize

import spacy
import neuralcoref
from spacy.symbols import nsubj, nsubjpass, VERB

from selenium.webdriver.common.keys import Keys
import chromedriver_binary

from dateutil.relativedelta import relativedelta
from datetime import datetime as dt

from selenium import webdriver
from bs4 import BeautifulSoup
from bs4.element import Tag
from random import randint
import winsound

nlp = spacy.load('en_core_web_lg')
nlp.max_length = 50000000

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\skamenshchikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Import CDFs:

In [3]:
##### HTML parsing #####
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

def get_unique_text(document):
    unique_sentences = []
    for sentence in [sent.raw for sent in TextBlob(document).sentences]:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return ' '.join(unique_sentences)

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
#####/HTML parsing #####

##### Text preprocessing #####
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

def text_normalize(txt):
    processed_text = (re.sub('[^a-zA-Z]', ' ', txt)).lower()
    processed_text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",processed_text)
    processed_text=re.sub("(\\d|\\W)+"," ",processed_text)

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(processed_text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]
    tokens = [i for i in tokens if (tags(i) in ['NN', 'NNP', 'NNS', 'NNPS'])]

    return tokens

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True
##### Text preprocessing #####

def readingTime(mytext):
    total_words = len(word_tokenize(mytext))
    estimatedTime = round(total_words/200.0,1)
    return estimatedTime

def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(60, 100)

def get_text(url):
    page = urlopen(url)
    soup = BeautifulSoup(page)
    fetched_text = ' '.join(map(lambda p:p.text,soup.find_all('p')))
    return fetched_text

def graph_keys(final_text, top_number):
    
    bigrams = list(nltk.ngrams(text_normalize(final_text),2))
    bigrams = [' '.join(i) for i in bigrams if (i[0]!=i[1])] 
    bigram_counts = collections.Counter(bigrams)
    
    df = pd.DataFrame(bigram_counts.most_common(len(bigram_counts)), columns=['bigram', 'count'])[:top_number]
    df['count'] = 100*df['count']/df['count'].sum().astype(int) 
    keys = ', '.join(list(df['bigram'].astype(str) + ' (' + df['count'].astype(int).astype(str) + '%)'))

    return keys

def tags(x):
    return nltk.pos_tag(nltk.word_tokenize(x))[0][1]

def syntax_full(spacy_sentence):
    result=[]
    for token in spacy_sentence:
        if (token.dep == nsubj or token.dep == nsubjpass) and token.head.pos == VERB:
            result.append(token.head)
    if result:
        return True
    else:
        return False

def filter_triplet(final_text):
    final_text = get_unique_text(final_text)
    doc = nlp(final_text)
    valid_sents=[]

    for s in list(doc.sents):
        if syntax_full(s):
            valid_sents.append(s.text)
    final_text = ' '.join(valid_sents)
    return final_text

def coref_res(rawtext, coref_greedn = 0.5):

    neuralcoref.add_to_pipe(nlp, greedyness = coref_greedn, store_scores=False)
    doc = nlp(rawtext)

    resolved = list(tok.text_with_ws for tok in doc)

    for cluster in doc._.coref_clusters:
        for coref in cluster:
            if coref != cluster.main:
                if coref.text[0].isalpha() and coref.text[0].isupper():

                    main_words_list=word_tokenize(cluster.main.text)
                    main_words_list[0]=main_words_list[0].capitalize()
                    resolved[coref.start] = detokenizer(main_words_list) + doc[coref.end-1].whitespace_

                for i in range(coref.start+1, coref.end):
                    resolved[i] = ""
            else:
                resolved[coref.start] = cluster.main.text + doc[coref.end-1].whitespace_
                for i in range(coref.start+1, coref.end):
                    resolved[i] = ""

    text_resolved = ''.join(resolved)
    nlp.remove_pipe("neuralcoref")

    return text_resolved

def check_min_num_of_clauses(spacy_sentence, n):
    result=[]
    for token in spacy_sentence:
        if (token.dep_ in ['nsubj','nsubjpass','csubj','expl']) and (token.head.pos_ == 'VERB' or token.head.pos_ == 'AUX'):
            result.append(token.head.text)
    if len(result)>=n:
        return True

    else:
        return False

def get_sents_ids_whitelist(spacy_sents):
    whitelist=[]
    i=1
    sents_texts=[]
    for sent in spacy_sents:
        if (sent.text not in sents_texts) and check_min_num_of_clauses(sent,1):
            whitelist.append(i)
            sents_texts.append(sent.text)
        i=i+1
    return(whitelist)

def compress(spacy_sents,sents_whitelist):
    blacklist_tokens=[]
    n=1
    for sent in spacy_sents:
        if (n in sents_whitelist):
            for token in sent:
                if token.dep_ in ['appos','advmod']:
                    token_sub_tree=token.subtree
                    for t in token_sub_tree:
                        blacklist_tokens.append(t.i)

        n=n+1
    return(blacklist_tokens)

def get_list_sents_tokens(spacy_sents,sents_whitelist,blacklist_tokens):
    sents_tokens=[]
    n=1
    for sent in spacy_sents:
        sent_tokens=[]
        if (n in sents_whitelist):
            for token in sent:
                if (token.i not in blacklist_tokens):
                    sent_tokens.append(token.text)
            sents_tokens.append(sent_tokens)
            sent_tokens=[]

        n=n+1
    return(sents_tokens)

def detokenizer(list_of_tokens):
    text_str="".join([" "+w if not w.startswith("'") and not w.startswith("’") and w!='' and w not in string.punctuation else w for w in list_of_tokens]).strip()
    return(text_str)

def sentence_grammar_fix(sentences):
    fixed=[]
    for sent in sentences:

        sent=sent.strip()
        sent=sent.replace('\n','')
        sent=sent.replace('()','')

        sent=re.sub('\s+',' ',sent)
        sent=sent+'.'
        sent=re.sub(r'([,.\-—:])+',r'\1',sent)

        if len(sent)>1:
            if sent[0] in ['.',',','-','—']:
                sent=sent[1:]
        sent=sent.strip()

        if len(sent)>1:
            if sent[0].isalpha():
                sent=sent[0].upper()+sent[1:]
        fixed.append(sent)

    return(fixed)

def spacy_compress(rawtext):

    doc1 = nlp(rawtext)
    sents_whitelist = get_sents_ids_whitelist(doc1.sents)

    tokens_blacklist = compress(doc1.sents,sents_whitelist)
    sents_tokens = get_list_sents_tokens(doc1.sents,sents_whitelist,tokens_blacklist)
    compressed_text_sents = []

    for s in sents_tokens:
        text=detokenizer(s)
        compressed_text_sents.append(text)
    compressed_text_sents=sentence_grammar_fix(compressed_text_sents)
    text =' '.join(compressed_text_sents)

    return(text)

############# Parse Wiki ############# 
def parse_wiki(google_url):
    
    # load driver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #/load driver
    
    # get urls  
    driver.get(google_url)
    time.sleep(randint(1,5))

    soup = BeautifulSoup(driver.page_source,'lxml')
    result_div = soup.find_all('div', attrs={'class': 'g'})

    links = []
    titles = []
    errors = []

    descriptions = []
    for r in result_div:
        try:
            link = r.find('a', href=True)
            title = None
            title = r.find('h3')

            if isinstance(title,Tag):
                title = title.get_text()

            description = None
            description = r.find('span', attrs={'class': 'st'})

            if isinstance(description, Tag):
                description = description.get_text()

            if link != '' and title != '' and description != '':
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)

        except Exception as e:
            print(e)
            continue

    url_list = links[:(page_number)]
    url_list = [i for i in url_list if 'https://en.wikipedia.org' in i]
    url_list = [i.split('/')[4] for i in url_list if 'https://en.wikipedia.org' in i]
     #/ get urls
        
    driver.stop_client()
    driver.close()
    
    return url_list     
############# Parse Wiki ############# 

############# Parse Arxiv #############
def parse_arxiv(query, delta_months):
    
    arxivtext = ''
    urls = []
    titles = []

    closest_value = 100
    req = 'https://arxiv.org/search/?query='+query+'&size='+str(closest_value)
    req = req + '&searchtype=all&source=header&start=0&date-filter_by=past_' + str(delta_months) 
    
    htmlString = get(req)

    soup = BeautifulSoup(htmlString.content, 'html5lib')
    hrefs = soup.find_all('a', {'href': re.compile(r'arxiv.org/abs/')})

    titles = list(soup.find_all('p', {'class' : 'title is-5 mathjax'}))[:page_number]
    titles_r = [i.text.replace('\n','').replace('  ','') for i in titles]
    titles = ', '.join(titles_r)

    if (len(hrefs) > 0):
        for i in hrefs:
            urls.append(i['href'])

    txt = []
    for i in urls[:page_number]:
        time.sleep(random.randint(1,8))
        soup = BeautifulSoup(get(str(i)).content, 'html5lib')
        abstract = ' '.join(soup.find('blockquote').text.replace('  ',' ').split())
        txt.append(abstract)

    arxivtext = re.sub('[^A-Za-z0-9.]+', ' ', '; '.join(txt))
    df = pd.DataFrame(list(zip(txt, urls, titles_r)), columns=['text','link', 'page'])

    return arxivtext, titles, df
#############/Parse Arxiv #############

def longest_common_substring(s1, s2):
  m = [[0] * (1 + len(s2)) for i in range(1 + len(s1))]
  longest, x_longest = 0, 0
  for x in range(1, 1 + len(s1)):
    for y in range(1, 1 + len(s2)):
      if s1[x - 1] == s2[y - 1]:
        m[x][y] = m[x - 1][y - 1] + 1
        if m[x][y] > longest:
          longest = m[x][y]
          x_longest = x
      else:
        m[x][y] = 0
  return s1[x_longest - longest: x_longest]

def longest_common_sentence(s1, s2):
    s1_words = s1.split(' ')
    s2_words = s2.split(' ')
    return ' '.join(longest_common_substring(s1_words, s2_words))

def css(a,b):
    if len(a.split()) > 0:
        score = len(longest_common_sentence(a,b).split())/len(a.split())
    else:    
        score = 0
    return score 
#/common string #

def parse_page(url,tag,cls=''): 
    
    htmlString = get(url).text
    soup = BeautifulSoup(htmlString, 'html.parser')
    paragraphs = soup.find(tag,cls)
    txt = text_from_html(str(paragraphs))

    return txt

############# Parse Google #############
def parse_google(query, keys_number, page_number, tag, D1, D2, cls=''):   
    
    # load driver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #/load driver

    # get urls
    timed_query = query +' after:' + str(D1) +' before:' + str(D2) 
    google_url = "https://www.google.com/search?q=" + timed_query + "&num=" + str(page_number+1)
    driver.get(google_url)
    time.sleep(randint(1,5))

    soup = BeautifulSoup(driver.page_source,'lxml')
    result_div = soup.find_all('div', attrs={'class': 'g'})

    links = []
    titles = []
    errors = []

    descriptions = []
    for r in result_div:
        try:
            link = r.find('a', href=True)
            title = None
            title = r.find('h3')

            if isinstance(title,Tag):
                title = title.get_text()

            description = None
            description = r.find('span', attrs={'class': 'st'})

            if isinstance(description, Tag):
                description = description.get_text()

            if link != '' and title != '' and description != '':
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)

        except Exception as e:
            print(e)
            continue

    url_list = links[:(page_number)]
    #/ get urls

    # load content 
    txt = []
    titles = []
    for j in tqdm(url_list):
        delta = random.randint(1,1)
        time.sleep(delta)
        
        try:  
            if str(j).endswith('.pdf'): 
                file_data = parser.from_file(str(j))           
                t = file_data['content']
            else:
                t = parse_page(j,tag,cls)
            txt.append(t)
            titles.append(''.join(sent_tokenize(t)[:3]))
        except:
            print('Parsing error:',str(j))
            errors.append(str(j))
       
    googletext = re.sub('[^A-Za-z0-9.]+', ' ', '; '.join(txt))
    df = pd.DataFrame(list(zip(txt, url_list, titles)), columns=['text','link', 'page'])
    
    driver.stop_client()
    driver.close()
   
    return googletext, errors, df
#############/Parse Google #############

############# Doc preparation ##########
def add_hyperlink(paragraph, text, url, flag):
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element and a new w:rPr element
    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    # Create a new Run object and add the hyperlink into it
    r = paragraph.add_run()
    r._r.append (hyperlink) 

    # A workaround for the lack of a hyperlink style (doesn't go purple after using the link)
    # Delete this if using a template that has the hyperlink style in it
    r.font.color.theme_color = MSO_THEME_COLOR_INDEX.HYPERLINK
    r.font.underline = flag

    return hyperlink


def save_doc(final_summary, summary, query, docs_number, readtime, url_keys, url_marks):
    
    sent_list = list(final_summary.split(sep='<hr>'))
    doc = Document()
    style = doc.styles['Normal']
    
    font = style.font
    font.name = 'Times New Roman'
    font.size = Pt(12)

    hd = doc.add_paragraph()
    hd.alignment = WD_ALIGN_PARAGRAPH.LEFT
    hd.add_run('Summary').bold = True

    if query != 'none':
        hd = doc.add_paragraph('Request: ' + "''" + query + "''")

    hd = doc.add_paragraph('Read time: ' + str(readtime) + ' min')
    hd = doc.add_paragraph('Documents: ' + str(docs_number))
    
    hd = doc.add_paragraph('')
    
    hd.add_run('Keys:\n').underline = True
    
    for j in url_keys:
        if j != url_keys[-1]:
            add_hyperlink(hd, (str(j.split('/keyword/')[1]) + ', '), str(j.split('/keyword/')[0]), False)
        else:
            add_hyperlink(hd, (str(j.split('/keyword/')[1])), str(j.split('/keyword/')[0]), False)
        
    hd.add_run('\n\nBenchmarks:\n').underline = True
    
    for j in url_marks:
        if j != url_marks[-1]:
            add_hyperlink(hd, str(j.split('/keyword/')[1]) + ', ', str(j.split('/keyword/')[0]), False)
        else:
            add_hyperlink(hd, str(j.split('/keyword/')[1]), str(j.split('/keyword/')[0]), False)
    
    r = hd.add_run()
    for i in sent_list:
        hd.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

        if query != 'none':
            try:
                link = re.search(r"<a href=(.*?)target='_blank'", str(i)).group(1).replace(' ','')
                hd = doc.add_paragraph(striphtml(str(i)).replace('<hr>','').replace('<u>','').replace('More',''))               
                add_hyperlink(hd, 'More', link, True).add_run()
            except:
                link = ''
        if query == 'none':
            hd = doc.add_paragraph(striphtml(str(i)).replace('<hr>','').replace('<u>','').replace('More',''))    
         
    doc.save('docs/' + summary + '.docx')
    
    return True
#############/Doc preparation ##########

def get_summary(rawtext, readtime):
    sentences = int(readtime/(np.median([len(i.split()) for i in nltk.sent_tokenize(rawtext)])/200))

    stemmer = Stemmer("english")
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words("english")
    parser = PlaintextParser.from_string(' '.join(sent_tokenize(rawtext)[6:]), Tokenizer("english"))

    text_list = []
    for sentence in summarizer(parser.document, sentences):
        text_list.append(str(sentence))

    txt = ' '.join(sent_tokenize(rawtext)[:6]) + ' '+' '.join(text_list)

    z = 0
    output = []
    pdf_output = []

    for i in nltk.sent_tokenize(txt):
        z = z+1
        output.append('\n\n<hr>' + str(z) + '. ' + str(i))
        pdf_output.append('<hr>' + str(z) + '. ' + str(i))

    txt = ''.join(output) + '<hr>'
    pdf_txt = ''.join(pdf_output) + '<hr>'
    return txt, pdf_txt

def get_entities(rawtext, tops):
    
    spacy_nlp = spacy.load('en_core_web_lg', disable=["tagger","parser"])
    doc = spacy_nlp(rawtext)

    ners = []
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'PERSON']:
            ners.append(ent.text)
   
    ner_counts = collections.Counter(ners)

    df = pd.DataFrame(ner_counts.most_common(len(ner_counts)), columns=['ner_names', 'count'])[:tops]
    df['count'] = 100*df['count']/df['count'].sum().astype(int) 
    keys = ', '.join(list(df['ner_names'].astype(str) + ' (' + df['count'].astype(int).astype(str) + '%)'))
    
    return keys

def get_jaccard_sim(a,b):

    a = set(a.split())
    b = set(b.split())
    c = a.intersection(b)

    return float(len(c)/len(a))

def filter_text(page_txt, page_sum, threshold=0.5): 
    sent_list = []  
    for j in page_txt.split('.'):
        try:
            sim_score = get_jaccard_sim(j, page_sum)
        except:
            sim_score = 0
            
        if sim_score > threshold:
            sent_list.append(j + '.')
            
    return sent_list        

Parameters:

In [5]:
UPLOAD_FOLDER = 'docs/'

readtime = 5

page_number = 10
keys_number = 10

process_time = 0
delta_months = 48

Create query:

In [6]:
query = input()

machine learning


Update query:

In [7]:
D1 = (dt.now() - relativedelta(months=delta_months)).strftime('%Y-%m-%d')
D2 = dt.now().strftime('%Y-%m-%d')

red_query = (query + " before:" + str(D2) + " after:" + str(D1))

Parse Wiki:

In [8]:
%%time

wikitext = ''
wikikeys = ''

df_wiki = pd.DataFrame()
wiki_wiki = wikipediaapi.Wikipedia('en', extract_format=wikipediaapi.ExtractFormat.WIKI)

red_query = "https://www.google.com/search?q=" + 'site:https://en.wikipedia.org ' + query + "&num=" + str(page_number+1)
wiki_titles = parse_wiki(red_query)

txts = []
titles = []

for i in tqdm(wiki_titles):  
    
    page_sum = wiki_wiki.page(i).summary
    page_txt = wiki_wiki.page(i).text
    sent_list = filter_text(page_txt, page_sum, threshold=0.5)
            
    titles.append(i)
    txts.append(' '.join(sent_list).replace('\n', ''))        
    
wikitext = ''.join(txts).replace('\n','')
wikikeys = graph_keys(wikitext, keys_number)
wiki_entities = get_entities(wikitext, keys_number)

url_list = [str('https://en.wikipedia.org/wiki/' + i)  for i in wiki_titles] 
df_wiki = pd.DataFrame(list(zip(txts, url_list, titles)), columns=['text','link', 'page'])

print('|Keywords|:', wikikeys, '\n')
print('|Entities|:', wiki_entities, '\n')
    
winsound.Beep(2500, 1000)    

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.20/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\skamenshchikov\.wdm\drivers\chromedriver\win32\87.0.4280.20]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  2.03it/s]


|Keywords|: machine algorithm (14%), computer vision (11%), deep network (11%), deep belief (11%), speech recognition (11%), belief network (9%), study computer (7%), field study (7%), computer science (7%), machine model (7%) 

|Entities|: ASR (18%), MDP (9%), Deep (9%), DefinitionDeep (9%), OverviewMost (9%), Boltzmann (9%), InterpretationsDeep (9%), Cresceptron (9%), Neocognitron (9%), ANNs (9%) 

Wall time: 38.9 s


Parse Arxiv:

In [9]:
%%time

arxivtext = ''
arxivkeys = ''

df_arxiv = pd.DataFrame()

try:
    arxiv_data = parse_arxiv(query, delta_months)
    arxivtext = arxiv_data[0]
    
    arxiv_entities = get_entities(arxivtext, keys_number)
    arxivkeys = graph_keys(arxivtext, keys_number)
    
    print('|Keywords|:', arxivkeys, '\n')
    print('|Entities|:', arxiv_entities, '\n')
    
    df_arxiv = arxiv_data[2]
except:
    print('No data')
    
winsound.Beep(2500, 1000)

|Keywords|: subclass label (13%), training data (13%), knowledge distillation (13%), action distribution (10%), classification task (10%), state art (10%), world agent (10%), abstract world (6%), hidden stratification (6%), image classification (6%) 

|Entities|: GEORGE (16%), LQG (11%), MFG (11%), ImageNet (11%), PermaKey (11%), Atari (11%), ICLR (11%), Generative Adversarial Networks (5%), Clustered Optimization of LAtent (5%), COLA (5%) 

Wall time: 1min 29s


Parse Google:

In [10]:
%%time

googletext = ''
googlekeys = ''

df_google = pd.DataFrame()
    
try:
    google_data = parse_google(query, keys_number, page_number, 'body', D1, D2)
    googletext = google_data[0]
        
    google_entities = get_entities(googletext, keys_number)
    googlekeys = graph_keys(googletext, keys_number)
    
    print('|Keywords|:', googlekeys, '\n')
    print('|Entities|:', google_entities, '\n')
  
    df_google = google_data[2]
except:
    print('No data')
    
winsound.Beep(2500, 1000)    

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\skamenshchikov\.wdm\drivers\chromedriver\win32\87.0.4280.20\chromedriver.exe] found in cache
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.81s/it]


|Keywords|: machine tool (12%), data fabric (12%), customer story (12%), quick link (11%), story try (11%), link data (10%), google cloud (9%), hybrid cloud (6%), cloud automl (5%), cloud service (5%) 

|Entities|: NetApp (24%), DZone (10%), GPU (10%), Watson (10%), ML (8%), Google Cloud (8%), Microsoft (8%), IBM (8%), GUI (5%), Scikit (5%) 

Wall time: 49.7 s


Parse Google Patents: 

In [11]:
%%time

patenttext = ''
patentkeys = ''

df_patent = pd.DataFrame()

try:
    z = parse_google("site:https://patents.google.com " + query, keys_number, page_number, 'div', D1, D2, 'abstract')
    patent_text = z[0] 
    
    patent_entities = get_entities(patent_text, keys_number)
    patentkeys = graph_keys(patent_text, keys_number)
    
    print('|Keywords|:', patentkeys, '\n')
    print('|Entities|:', patent_entities, '\n')
    
    df_patent = z[2]
except:
    print('No data')
    
df_patent.replace('', np.nan, inplace=True)
df_patent.dropna(inplace=True)  

winsound.Beep(2500, 1000)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\skamenshchikov\.wdm\drivers\chromedriver\win32\87.0.4280.20\chromedriver.exe] found in cache
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.03s/it]


|Keywords|: body feature (19%), antibody affinity (13%), client device (11%), affinity antigen (8%), fault machine (8%), determination data (8%), cognitive intelligence (8%), update matrix (8%), measurement body (8%), method system (5%) 

|Entities|: GPU (75%), IoT (25%) 

Wall time: 51.3 s


Trend keys:

In [71]:
%%time

a = (arxivtext + patent_text) 
b = (googletext + wikitext) 

specific_keys = get_entities(a, keys_number)
common_keys = get_entities(b, keys_number)

specific_keys = [re.sub(r" ?\([^)]+\)", "", i).strip() for i in specific_keys.split(',')]
common_keys = [re.sub(r" ?\([^)]+\)", "", i).strip() for i in common_keys.split(',')]

df1 = pd.DataFrame(list(zip(specific_keys)), columns =['specific'])
df2 = pd.DataFrame(list(zip(common_keys)), columns =['known'])

matches = fpd.fuzzy_merge(df1, df2, left_on=['specific'], right_on=['known'], ignore_case=True, keep='match', 
                         method='levenshtein', threshold=0.8, join='inner')

specific_reqs = list(set(specific_keys).difference(set(list(matches['specific']))))
specific_reqs = [i for i in specific_reqs if i[0].isupper()] + [i for i in specific_reqs if i[0].islower()]
specific_keys = ', '.join([i for i in specific_reqs if i])

print('|Trend keys|:', specific_keys, '\n')

winsound.Beep(2500, 1000)

|Trend keys|: DenseNet, LC KSVD, MIAS, CNN, RCNN, Recursive Convolutional Neural Network R, BreakHis, RNA, CAD, CMS 

Wall time: 25.1 s


Concatenate dataframes:

In [12]:
df = df_wiki.append(df_google).append(df_arxiv).append(df_patent)

Syntactic filtering:

In [14]:
%%time

final_text = wikitext + googletext + arxivtext + patenttext

final_text = filter_triplet(final_text)
final_text = coref_res(final_text)

winsound.Beep(2500, 1000)

Wall time: 19 s


Get summary and tags:

In [15]:
%%time

report_summary = get_summary(final_text, readtime)[1]

final_entities = get_entities(final_text, keys_number)
final_keys = graph_keys(final_text, keys_number)

winsound.Beep(2500, 1000)

Wall time: 25 s


Create keys with urls:

In [16]:
url_keys = []
for i in final_keys.split(','):
    url = 'https://www.google.com/search?q=' + '+'.join(re.sub(r" ?\([^)]+\)", "", i).strip().split()) + '+' + query + '/keyword/' + i 
    url_keys.append(url) 
    
mark_keys = []
for i in final_entities.split(','):
    url = 'https://www.google.com/search?q=' + '+'.join(re.sub(r" ?\([^)]+\)", "", i).strip().split()) + '+' + query + '/keyword/' + i
    mark_keys.append(url)     

Add urls:

In [17]:
ref_list = []
pdf_list = []

sent_list = list(report_summary.split(sep='<hr>'))[1:]

for i in sent_list[:-1]:
    try:
        df['score'] = df['text'].apply(lambda x: css(i,x))
        df = df.sort_values(by=['score'], ascending=False)
        
        if str(df['link'].iloc[0]):
            pdf_list.append(str(str(i) + " <u><a href=" + str(df['link'].iloc[0]) + " target='_blank'>" + "More" + "</a></u>" + "<hr>"))
            ref_list.append(str(df['link'].iloc[0]))
    except:
        pdf_list.append('')

ref_list = list(frozenset(ref_list))
pdf_summary = ''.join(pdf_list)

Save docx:

In [18]:
save_doc(pdf_summary, 'summary', query, len(df), readtime, url_keys, mark_keys)

True

Statistics:

In [19]:
memory_size = int(len(final_text.encode('utf-8'))/1024)
docs_number = len(df)

print('Memory size:', memory_size, 'mb')
print('Number of documents:', docs_number)

winsound.Beep(2500, 1000)

Memory size: 33 mb
Number of documents: 34
